# Setup Code
**DO NOT CHANGE THIS!!**
**RUN THE CELL BELOW FIRST!!**

In [1]:
import carla
import os

client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
if client_name == "DOES NOT EXIST":
    raise Exception("The environment variable for the container name of the carla server has not been set")

# Connect to the client and retrieve the world object
client = carla.Client(client_name, 2000)
client.set_timeout(100.0)
print(client.get_available_maps())
# map_name = '/Game/Carla/Maps/Town10HD' # Example map name, CARLA supports Town01, Town02, etc.
# client.load_world(map_name)
# Get the world
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)
print("Got world")

['/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town10HD_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town10HD']
Got world


In [2]:
print(world)

World(id=13993395647333372683)


# Start Adding Code or Markdown Cells Below
Some helpful code cells are provided below that you may choose to use

In [ ]:
#### SPAWN VEHICLES ####
"""
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points)))
"""
#### SET ALL VEHICLES TO AUTOPILOT
"""
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)
"""

In [ ]:
!pip install pillow

In [3]:
import time
import random

SAVE_DIR = "sim_images"

weather = {
  "ClearNoon": carla.WeatherParameters.ClearNoon,
  "CloudyNoon": carla.WeatherParameters.CloudyNoon,
  "WetNoon": carla.WeatherParameters.WetNoon,
  "WetNoon": carla.WeatherParameters.WetCloudyNoon,
  "WetNoon": carla.WeatherParameters.MidRainyNoon,
  "HardRainNoon": carla.WeatherParameters.HardRainNoon,
  "SoftRainNoon": carla.WeatherParameters.SoftRainNoon,
  "ClearSunset": carla.WeatherParameters.ClearSunset,
  "CloudySunset": carla.WeatherParameters.CloudySunset,
  "WetSunset": carla.WeatherParameters.WetSunset,
  "WetCloudySunset": carla.WeatherParameters.WetCloudySunset,
  "MidRainSunset": carla.WeatherParameters.MidRainSunset,
  "HardRainSunset": carla.WeatherParameters.HardRainSunset,
  "SoftRainSunset": carla.WeatherParameters.SoftRainSunset
  }

def save_image(image, image_type, depth=False):
    """
    Saves the image to a file.
    """
    print("Saving image")
    if depth:
        image.save_to_disk('%s/%06d_%s.png' % (SAVE_DIR, image.frame, image_type), carla.ColorConverter.LogarithmicDepth)
    else:
        image.save_to_disk('%s/%06d_%s.png' % (SAVE_DIR, image.frame, image_type))
def main():
    actor_list = []
    print("Starting main")
    try:

        print("Set settings")
        blueprint_library = world.get_blueprint_library()
        
        traffic_manager = client.get_trafficmanager(8000) # Default port for Traffic Manager
        traffic_manager.set_global_distance_to_leading_vehicle(2.0)
        traffic_manager.set_synchronous_mode(True)
        
        # Find the Tesla Model 3 blueprint
        tesla_model_3_bp = blueprint_library.find('vehicle.tesla.model3')
        
        # Choose a spawn point
        spawn_point = random.choice(world.get_map().get_spawn_points())
        
        # Spawn the Tesla
        car = world.spawn_actor(tesla_model_3_bp, spawn_point)
        
        # Set up the camera blueprint
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        
        blueprint_library = world.get_blueprint_library()
        cam_bp = blueprint_library.find('sensor.camera.rgb')
        cam_bp.set_attribute('image_size_x', '1920')
        cam_bp.set_attribute('image_size_y', '1200')
        cam_bp.set_attribute('fov', '120')
        # cam_bp.set_transform(camera_transform)

        # Set up the depth camera blueprint
        depth_cam_bp = blueprint_library.find('sensor.camera.depth')
        depth_cam_bp.set_attribute('image_size_x', '1920')
        depth_cam_bp.set_attribute('image_size_y', '1200')
        depth_cam_bp.set_attribute('fov', '120')
        # depth_cam_bp.set_transform(camera_transform)

        # Spawn the camera sensor
        camera_sensor = world.spawn_actor(cam_bp, camera_transform, attach_to=car)
        actor_list.append(camera_sensor)

        # Spawn the depth camera sensor
        depth_camera_sensor = world.spawn_actor(depth_cam_bp, camera_transform, attach_to=car)
        actor_list.append(depth_camera_sensor)
        print("Spawned sensors")
        # Register the callback to save images
        camera_sensor.listen(lambda image: save_image(image, 'camera'))
        depth_camera_sensor.listen(lambda image: save_image(image, 'depth'))

        # Measure and log FPS
        start_time = time.time()
        frames = 0
        print("Starting simulation")
        car.set_autopilot(True)
        while True:
            world.tick()
            frames += 1
            elapsed_time = time.time() - start_time
            print(frames)
            time.sleep(0.5)
            # if elapsed_time > 1.0:  # log every second
            #     fps = frames / elapsed_time
            #     print("FPS:", fps)
            #     frames = 0
            #     start_time = time.time()
    except Exception as e:
        # This will catch any exception that occurs within the try block
        print(f'An error occurred: {e}')
    finally:
        
        print('destroying actors')
        for actor in actor_list:
            actor.destroy()
        print('done.')
main()

Starting main
Set settings
Spawned sensors
Starting simulation
1
Saving image
Saving image
2
Saving image
3
Saving image
4
Saving image
Saving image
5
Saving image
6
Saving image
Saving image
7
8
Saving image
Saving image
9
Saving image
Saving image
10
Saving image
11
Saving image
12
Saving image
13
Saving image
Saving image
14
Saving image
Saving image
15
Saving image
16
Saving image
17
Saving image
18
Saving image
Saving image
19
Saving image
Saving image
20
21
Saving image
Saving image
22
Saving image
Saving image
23
Saving image
24
Saving image
25
Saving image
26
Saving image
Saving image
27
Saving image
Saving image
28
29
Saving image
Saving image
30
Saving image
31
Saving image
Saving image
32
Saving image
33
Saving image
34
Saving image
Saving image
35
Saving image
Saving image
36
37
Saving image
Saving image
38
Saving image
39
Saving image
Saving image
40
Saving image
41
Saving image
42
Saving image
Saving image
43
Saving image
Saving image
44
45
Saving image
Saving image
46
Sa

KeyboardInterrupt: 

In [5]:
!apt update
!apt-get install -y libglib2.0-0 libsm6 libxrender1 libxext6

Hit:1 http://deb.debian.org/debian bullseye InRelease
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [270 kB]
Fetched 363 kB in 3s (133 kB/s)   
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxext6 is already the newest version (2:1.3.3-1.1).
libxext6 set to manually installed.
The following additional packages will be installed:
  libglib2.0-data libice6 libicu67 libxml2 shared-mime-info x11-common
  xdg-user-dirs
The following NEW packages will be installed:
  libglib2.0-0 libglib2.0-data libice6 libicu67 libsm6 libxml2 libxrender1
  shared-mime-info x11-common xdg-user-dirs
0 upgra

In [7]:
import cv2
import numpy as np
import os
from glob import glob

# Directory containing images
image_folder = 'sim_images'
# Output video file
video_file = 'sim_depth_video.mp4'

images = sorted(glob(f"{image_folder}/*_depth.png"))

# Determine the width and height from the first image
image_path = images[0]
frame = cv2.imread(image_path)
height, width, layers = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(video_file, fourcc, 20.0, (width, height))

for i,image in enumerate(images):
    img = cv2.imread(image)
    print(i, "out of", len(images))
    out.write(img)  # Write out frame to video

out.release()  # Release the video writer

# Check if video file has been created and return appropriate response
if os.path.exists(video_file):
    video_file_path = video_file
else:
    video_file_path = "Video conversion failed!"

video_file_path

0 out of 152
1 out of 152
2 out of 152
3 out of 152
4 out of 152
5 out of 152
6 out of 152
7 out of 152
8 out of 152
9 out of 152
10 out of 152
11 out of 152
12 out of 152
13 out of 152
14 out of 152
15 out of 152
16 out of 152
17 out of 152
18 out of 152
19 out of 152
20 out of 152
21 out of 152
22 out of 152
23 out of 152
24 out of 152
25 out of 152
26 out of 152
27 out of 152
28 out of 152
29 out of 152
30 out of 152
31 out of 152
32 out of 152
33 out of 152
34 out of 152
35 out of 152
36 out of 152
37 out of 152
38 out of 152
39 out of 152
40 out of 152
41 out of 152
42 out of 152
43 out of 152
44 out of 152
45 out of 152
46 out of 152
47 out of 152
48 out of 152
49 out of 152
50 out of 152
51 out of 152
52 out of 152
53 out of 152
54 out of 152
55 out of 152
56 out of 152
57 out of 152
58 out of 152
59 out of 152
60 out of 152
61 out of 152
62 out of 152
63 out of 152
64 out of 152
65 out of 152
66 out of 152
67 out of 152
68 out of 152
69 out of 152
70 out of 152
71 out of 152
72

'sim_depth_video.mp4'

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Replace this with the path to your video file
video_path = 'sim_video.mp4'

# Encode the video to base64 and display it in the notebook
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)